In [ ]:
------------GEO Readiness & Governance--------------
4TH SUB PILLARS

In [ ]:
--------------------Domain Trust Signals-------------------------------
** GEO Readiness & Governance → Domain Trust Signals

Sub-Pillar 4: Domain Trust Signaling Infrastructure**

Goal: “Can AI agents trust your domain as an authoritative, stable, and machine-verifiable source?”

This sub-pillar ensures your domain emits verifiable authenticity signals that modern AI agents require to trust,
rank, and respond using your content.

Sub-Pillar 4: Domain Trust Signaling Infrastructure
Ensures your domain emits clear, machine-readable trust signals that modern AI agents depend on.
It protects your content, improves AI visibility, stabilizes metadata, and confirms your domain identity to all automated systems.

In [ ]:
---------what is current code doing -----------------------
code covers HTTPS + SSL + Security Headers + DNS Identity Records — but Sub-pillar 1.6
includes more trust signals that your code does NOT include at all.

Sub-pillar 6 = Domain security + Identity + Machine-readable trust
code only covers:

basic HTTPS

partial SSL check

partial headers

2 DNS records

In [2]:
# Pillar 1, Sub-pillar 6
# See Bridge.ipynb cell 6 for logic
# ...existing code...
import requests
import ssl
import socket
from datetime import datetime
from urllib.parse import urlparse
try:
    import dns.resolver
except ImportError:
    print("Please install dnspython: pip install dnspython")

# Helper to print colored and formatted text for better readability
def print_header(text):
    print("\n" + "="*70)
    print(f" {text}")
    print("="*70)

def print_subheader(text):
    print("\n" + "-"*70)
    print(f" {text}")
    print("-"*70)

def print_status(message, status):
    padded_message = f"{message:<55}"
    if status == "FAIL" or status == "CRITICAL":
        status_str = f"[\033[91m{status}\033[0m]" # Red
    elif status == "WARN":
        status_str = f"[\033[93m{status}\033[0m]" # Yellow
    elif status == "PASS" or status == "INFO":
        status_str = f"[\033[92m{status}\033[0m]" # Green
    else:
        status_str = f"[{status}]"
    print(f"{padded_message} {status_str}")

def print_recommendation(rec):
    print(f"  - {rec}")

class DomainTrustAnalyzer:
    """
    Analyzes a domain's security and trust signals based on ARI v10.0 Pillar 1, Sub-pillar 6.
    """
    def __init__(self, target_url):
        if not target_url.startswith('http'):
            target_url = 'https://' + target_url
        self.url = target_url
        self.parsed_url = urlparse(self.url)
        self.domain = self.parsed_url.netloc
        self.report = {
            "checks": {}, "recommendations": [], "score": 0, "status": "Not Assessed"
        }
        self.session = requests.Session()
        self.session.headers.update({'User-Agent': 'ARI-DomainTrust-Analyzer/1.0'})

    def check_https_enforcement(self):
        print_subheader("1. HTTPS Enforcement")
        # Check 1: Strict Redirect from HTTP to HTTPS
        http_url = self.url.replace('https://', 'http://')
        try:
            res = self.session.head(http_url, allow_redirects=True, timeout=5)
            if res.url.startswith('https://'):
                self.report["checks"]["http_redirect"] = True
                print_status("HTTP requests redirect to HTTPS", "PASS")
            else:
                self.report["checks"]["http_redirect"] = False
                print_status("HTTP does not redirect to HTTPS", "FAIL")
                self.report["recommendations"].append("Implement a server-side 301 redirect from HTTP to all pages.")
        except requests.exceptions.RequestException:
            self.report["checks"]["http_redirect"] = True # If HTTP fails to connect, it's effectively enforced
            print_status("HTTP port is not open (good)", "PASS")

        # Check 2: SSL Certificate Validity
        try:
            # The request itself will fail on bad certs, but we can get expiry info manually
            context = ssl.create_default_context()
            with socket.create_connection((self.domain, 443)) as sock:
                with context.wrap_socket(sock, server_hostname=self.domain) as ssock:
                    cert = ssock.getpeercert()
            self.report["checks"]["ssl_valid"] = True
            print_status("SSL Certificate is trusted", "PASS")

            # Check Expiry
            expiry_date = datetime.strptime(cert['notAfter'], '%b %d %H:%M:%S %Y %Z')
            days_left = (expiry_date - datetime.utcnow()).days
            if days_left < 14:
                 print_status(f"Certificate expires in {days_left} days", "WARN")
                 self.report["recommendations"].append("Renew the SSL certificate soon.")
            else:
                 print_status(f"Certificate is valid for {days_left} more days", "PASS")

        except (ssl.SSLError, socket.gaierror, ConnectionRefusedError) as e:
            self.report["checks"]["ssl_valid"] = False
            print_status(f"SSL Certificate is invalid or untrusted: {e}", "FAIL")
            self.report["recommendations"].append("Install a valid, trusted SSL certificate from a known CA.")

    def check_security_headers(self):
        print_subheader("2. Security Headers")
        try:
            res = self.session.get(self.url, timeout=5)
            headers = res.headers

            # Check for HSTS
            if 'Strict-Transport-Security' in headers:
                self.report["checks"]["hsts_header"] = True
                print_status("Strict-Transport-Security (HSTS) header", "PASS")
            else:
                self.report["checks"]["hsts_header"] = False
                print_status("Strict-Transport-Security (HSTS) header", "FAIL")
                self.report["recommendations"].append("Implement the HSTS header to prevent downgrade attacks.")

            # Check for CSP
            if 'Content-Security-Policy' in headers:
                self.report["checks"]["csp_header"] = True
                print_status("Content-Security-Policy (CSP) header", "PASS")
            else:
                self.report["checks"]["csp_header"] = False
                print_status("Content-Security-Policy (CSP) header", "WARN")
                self.report["recommendations"].append("Implement a CSP to mitigate XSS and other injection attacks.")

            # Check for X-Content-Type-Options
            if headers.get('X-Content-Type-Options', '').lower() == 'nosniff':
                self.report["checks"]["x_content_type_header"] = True
                print_status("X-Content-Type-Options header is 'nosniff'", "PASS")
            else:
                self.report["checks"]["x_content_type_header"] = False
                print_status("X-Content-Type-Options header", "WARN")
                self.report["recommendations"].append("Set the X-Content-Type-Options header to 'nosniff'.")

        except requests.exceptions.RequestException:
             print_status("Could not fetch headers from the domain", "FAIL")

    def check_domain_identity_records(self):
        print_subheader("3. Domain Identity & Trust (DNS Records)")
        base_domain = self.domain.replace('www.', '')
        # Check for CAA
        try:
            dns.resolver.resolve(base_domain, 'CAA')
            self.report["checks"]["caa_record"] = True
            print_status("Certification Authority Authorization (CAA) record", "PASS")
        except (dns.resolver.NoAnswer, dns.resolver.NXDOMAIN):
            self.report["checks"]["caa_record"] = False
            print_status("Certification Authority Authorization (CAA) record", "WARN")
            self.report["recommendations"].append("Add a CAA DNS record to specify which CAs can issue certificates.")

        # Check for DMARC
        try:
            dns.resolver.resolve(f'_dmarc.{base_domain}', 'TXT')
            self.report["checks"]["dmarc_record"] = True
            print_status("DMARC email authentication record", "PASS")
        except (dns.resolver.NoAnswer, dns.resolver.NXDOMAIN):
            self.report["checks"]["dmarc_record"] = False
            print_status("DMARC email authentication record", "WARN")
            self.report["recommendations"].append("Add a DMARC DNS record to prevent email spoofing and phishing.")

    def run_analysis(self):
        print_header("ARI Sub-Pillar 1.6: Domain Security & Identity Trust")
        self.check_https_enforcement()
        self.check_security_headers()
        self.check_domain_identity_records()
        self._generate_final_report()
        self._print_final_report()

    def _generate_final_report(self):
        score = 0
        if self.report["checks"].get("http_redirect"): score += 20
        if self.report["checks"].get("ssl_valid"): score += 20
        if self.report["checks"].get("hsts_header"): score += 15
        if self.report["checks"].get("csp_header"): score += 10
        if self.report["checks"].get("x_content_type_header"): score += 5
        if self.report["checks"].get("caa_record"): score += 15
        if self.report["checks"].get("dmarc_record"): score += 5

        self.report["score"] = min(100, score)

        if self.report["score"] >= 90: self.report["status"] = "Excellent"
        elif self.report["score"] >= 70: self.report["status"] = "Good"
        elif self.report["score"] >= 40: self.report["status"] = "Needs Improvement"
        else: self.report["status"] = "Poor"

    def _print_final_report(self):
        print_header("Final Assessment Report")
        print_status("Overall Status", self.report['status'])
        print_status("ARI Score (out of 100)", self.report['score'])

        if self.report["recommendations"]:
            print_subheader("Recommendations")
            for rec in self.report["recommendations"]:
                print_recommendation(rec)


Please install dnspython: pip install dnspython


In [3]:
# main program 
target_url = input("Enter Input ::: ")
analyzer = DomainTrustAnalyzer(target_url)
analyzer.run_analysis()   

Enter Input :::  https://buildbridges.co/



 ARI Sub-Pillar 1.6: Domain Security & Identity Trust

----------------------------------------------------------------------
 1. HTTPS Enforcement
----------------------------------------------------------------------
HTTP requests redirect to HTTPS                         [PASS]
SSL Certificate is trusted                              [PASS]
Certificate is valid for 79 more days                   [PASS]

----------------------------------------------------------------------
 2. Security Headers
----------------------------------------------------------------------
Strict-Transport-Security (HSTS) header                 [FAIL]
Content-Security-Policy (CSP) header                    [WARN]
X-Content-Type-Options header                           [WARN]

----------------------------------------------------------------------
 3. Domain Identity & Trust (DNS Records)
----------------------------------------------------------------------


/tmp/ipykernel_104956/3091457280.py:85: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  days_left = (expiry_date - datetime.utcnow()).days


AttributeError: 'NoneType' object has no attribute 'resolver'

In [ ]:
-----------WHAT IS MISSING IN  CURRENT IMPLEMENTATION (exact and specific) -----------

Sub-Pillar 1.6 = Domain Security & Identity Trust Layer.

Your code covers only ~30% of required checks.

Below is exactly what is missing in THIS file:

1️⃣ Missing SSL / Certificate Trust Checks

    ONLY check:
    
        1. Certificate validity
        
        2. Expiry date
    
    Missing required checks:
    
        1. Not checking the certificate SAN (SubjectAltName)
    
    → Must match domain.
    
        1.Not checking certificate chain
    
    → only see leaf cert. No:
    
        chain validation
    
        issuer validation
    
        root trust store
    
        intermediate CA presence
    
        Not checking OCSP stapling
    
    → Required for real-time revocation.
    
        Not checking Certificate Transparency (SCT)
    
    → Required in SP-1.6.
    
        Not checking DNSSEC
    
    → Part of identity trust.
    
    So yes — we need cryptography library for proper parsing.

2️⃣ Missing DNS-Based Identity Signals

    DNS exceptions unhandled


    Timeout

    NoNameservers

    DNSException

    YXDOMAIN

    FormError

3️⃣ Missing Security Headers (you only check 3 of 10 mandatory ones)

     check:
    
    ✔️ HSTS
    ✔️ CSP
    ✔️ X-Content-Type

    Missing required headers:
    
    ❌ Referrer-Policy
    ❌ Permissions-Policy
    ❌ X-Frame-Options
    ❌ X-XSS-Protection
    ❌ Cross-Origin-Resource-Policy (CORP)
    ❌ Cross-Origin-Opener-Policy (COOP)
    ❌ Cross-Origin-Embedder-Policy (COEP)

    Sub-pillar 6 requires all of them.

4️⃣ Missing Robots/Policy Discovery
    
    Your code ignores machine-readable trust files:
    
    ❌ robots.txt
    ❌ ai-robots.txt
    ❌ ai.txt
    ❌ llm.txt
    ❌ /.well-known/ai.json
    ❌ /.well-known/llm-policy.json
    ❌ manifestai.json
    
    These are explicitly part of GEO Trust Signals.
    
    Right now your code cannot determine if agents are allowed or safe to access your domain, which is the whole point of this sub-pillar.

5️⃣ Missing Infrastructure / Hosting Identity

    Sub-pillar 6 requires trust signals about:
    
    ❌ IP address
    ❌ ASN
    ❌ Hosting provider (AWS, GCP, Cloudflare, Azure, etc.)
    ❌ IP reputation
    ❌ Geolocation mismatches (e.g., domain country vs. server)

You have none of these.

6️⃣ Missing WHOIS-based Domain Identity

 currently do ZERO ownership checks:

    ❌ Domain age
    ❌ Domain expires soon
    ❌ Registrar reputation
    ❌ WHOIS privacy vs. public
    ❌ Suspicious TLD / domain patterns

    These are explicitly required for identity trust.

7️⃣ Missing Availability & Stability Trust Signals

Your code does not check:

    ❌ Multiple HTTP checks (uptime)
    ❌ HEAD consistency
    ❌ Rate-limit detection (429)
    ❌ Retry-After handling
    ❌ HTTP/2 or HTTP/3 support
    ❌ Brotli/GZIP compression

Sub-pillar 6 requires verifying the technical trustworthiness of access.

8️⃣ Missing GEO Trust Output

    final output gives:
    
    score
    
    status
    
    recommendations
    
    Missing:
    
    ❌ GEO Trust Tier (T0–T4)
    ❌ Machine-readable declaration score
    ❌ Identity confidence score
    ❌ Infrastructure trust score
    ❌ Final “Agent Ready / Not Ready”

Your Score = ARI
GEO needs two-layer scoring.
--------- SUMMARY (short + direct) -------------
current code is missing 29 required checks.



Sub-pillar 6 = Domain security + Identity + Machine-readable trust
Your code only covers:

basic HTTPS

partial SSL check

partial headers

2 DNS records

The rest (about 70%) is not implemented.

In [ ]:
# -----------IMPROVED VERSION -----------------

In [17]:

"""
Patched DomainTrustAnalyzer (A1-Full patch) — minimal changes to  original file,
adds broader checks, defensive DNS handling, and uses cryptography for robust cert parsing.
Includes verbose DEBUG prints for tracing behavior.
"""
import requests
import ssl
import socket
from datetime import datetime, timezone
from urllib.parse import urlparse
import sys

# optional imports — handled gracefully
try:
    import dns.resolver
    import dns.exception
    DNSPYTHON_AVAILABLE = True
except Exception:
    dns = None
    dns = None
    DNSPYTHON_AVAILABLE = False

try:
    from cryptography import x509
    from cryptography.hazmat.primitives import serialization
    from cryptography.hazmat.backends import default_backend
    CRYPTO_AVAILABLE = True
except Exception:
    x509 = None
    CRYPTO_AVAILABLE = False

# Helper to print colored and formatted text for better readability
def print_header(text):
    print("\n" + "="*70)
    print(f" {text}")
    print("="*70)


def print_subheader(text):
    print("\n" + "-"*70)
    print(f" {text}")
    print("-"*70)


def print_status(message, status):
    padded_message = f"{message:<55}"
    if status == "FAIL" or status == "CRITICAL":
        status_str = f"[\033[91m{status}\033[0m]" # Red
    elif status == "WARN":
        status_str = f"[\033[93m{status}\033[0m]" # Yellow
    elif status == "PASS" or status == "INFO":
        status_str = f"[\033[92m{status}\033[0m]" # Green
    else:
        status_str = f"[{status}]"
    print(f"{padded_message} {status_str}")


def print_recommendation(rec):
    print(f"  - {rec}")


class DomainTrustAnalyzer:
    """
    Patched DomainTrustAnalyzer: retains original structure but adds checks and robustness.
    Minimal API changes: still initialize with a target_url and call run_analysis().
    """
    def __init__(self, target_url):
        if not target_url.startswith('http'):
            target_url = 'https://' + target_url
        self.url = target_url
        self.parsed_url = urlparse(self.url)
        self.domain = self.parsed_url.netloc
        # if netloc includes port, strip it for TLS/DNS queries
        if ':' in self.domain:
            self.domain = self.domain.split(':')[0]
        self.report = {
            "checks": {}, "recommendations": [], "score": 0, "status": "Not Assessed",
            "debug": {}
        }
        self.session = requests.Session()
        self.session.headers.update({'User-Agent': 'ARI-DomainTrust-Analyzer/1.0'})

    # ---------------- HTTPS & Certs ----------------
    def check_https_enforcement(self):
        print_subheader("1. HTTPS Enforcement & Certificate")
        # HTTP -> HTTPS redirect check
        http_url = self.url.replace('https://', 'http://')
        try:
            res = self.session.head(http_url, allow_redirects=True, timeout=6)
            redirected_to = res.url
            self.report['debug']['http_head_final_url'] = redirected_to
            if redirected_to.startswith('https://'):
                self.report["checks"]["http_redirect"] = True
                print_status("HTTP requests redirect to HTTPS", "PASS")
            else:
                self.report["checks"]["http_redirect"] = False
                print_status("HTTP does not redirect to HTTPS", "FAIL")
                self.report["recommendations"].append("Implement a server-side 301 redirect from HTTP to HTTPS for all pages.")
        except requests.exceptions.RequestException as e:
            # Connection failed on HTTP — often port closed; treat as enforced but flag for debug
            self.report["checks"]["http_redirect"] = True
            self.report['debug']['http_head_exception'] = str(e)
            print_status("HTTP port is not open or HEAD failed (treated as enforced)", "PASS")

        # TLS certificate retrieval and parsing — prefer cryptography if available
        try:
            context = ssl.create_default_context()
            with socket.create_connection((self.domain, 443), timeout=6) as sock:
                with context.wrap_socket(sock, server_hostname=self.domain) as ssock:
                    der_cert = ssock.getpeercert(binary_form=True)
                    # save raw cert for debug
                    self.report['debug']['cert_binary_len'] = len(der_cert) if der_cert else None

            if not der_cert:
                raise ValueError("Empty certificate")

            if CRYPTO_AVAILABLE:
                cert = x509.load_der_x509_certificate(der_cert, backend=default_backend())
                self.report["checks"]["ssl_valid"] = True
                print_status("SSL Certificate retrieved and parsed", "PASS")

                # expiry using cryptography (not brittle string parsing)
                expiry_date = cert.not_valid_after_utc
                # ensure timezone-aware for comparison
                if expiry_date.tzinfo is None:
                    expiry_date = expiry_date.replace(tzinfo=timezone.utc)
                days_left = (expiry_date - datetime.now(timezone.utc)).days
                self.report['debug']['cert_not_valid_after'] = expiry_date.isoformat()

                if days_left < 0:
                    print_status(f"Certificate expired {abs(days_left)} days ago", "FAIL")
                    self.report["recommendations"].append("Replace the expired SSL certificate immediately.")
                elif days_left < 14:
                    print_status(f"Certificate expires in {days_left} days", "WARN")
                    self.report["recommendations"].append("Renew the SSL certificate soon.")
                else:
                    print_status(f"Certificate is valid for {days_left} more days", "PASS")

                # SANs
                try:
                    sans = []
                    ext = cert.extensions.get_extension_for_class(x509.SubjectAlternativeName)
                    sans = ext.value.get_values_for_type(x509.DNSName)
                    self.report['debug']['certificate_sans'] = sans
                    print_status(f"Certificate SANs: {sans}", "DEBUG")
                except Exception:
                    self.report['debug']['certificate_sans'] = None

                # Certificate transparency and OCSP stapling checks could be added here (requires additional network calls)

            else:
                # Fallback to ssl.getpeercert() parsing — less robust but used when cryptography missing
                context = ssl.create_default_context()
                with socket.create_connection((self.domain, 443), timeout=6) as sock:
                    with context.wrap_socket(sock, server_hostname=self.domain) as ssock:
                        certinfo = ssock.getpeercert()
                if certinfo:
                    self.report["checks"]["ssl_valid"] = True
                    print_status("SSL Certificate is trusted (parsed via stdlib)", "PASS")
                    # try to parse notAfter if present
                    not_after = certinfo.get('notAfter')
                    self.report['debug']['cert_notAfter_raw'] = not_after
                    try:
                        expiry_date = datetime.strptime(not_after, '%b %d %H:%M:%S %Y %Z')
                        days_left = (expiry_date - datetime.utcnow()).days
                        if days_left < 14:
                            print_status(f"Certificate expires in {days_left} days", "WARN")
                            self.report["recommendations"].append("Renew the SSL certificate soon.")
                        else:
                            print_status(f"Certificate is valid for {days_left} more days", "PASS")
                    except Exception:
                        print_status("Could not parse certificate expiry reliably", "WARN")
                else:
                    raise ssl.SSLError("Empty cert info from stdlib")

        except Exception as e:
            # broad catch to avoid crashing the whole analyzer
            self.report["checks"]["ssl_valid"] = False
            self.report['debug']['ssl_exception'] = str(e)
            print_status(f"SSL Certificate is invalid or could not be retrieved: {e}", "FAIL")
            self.report["recommendations"].append("Install a valid, trusted SSL certificate from a known CA and ensure port 443 is reachable.")

    # ---------------- Security Headers ----------------
    def check_security_headers(self):
        print_subheader("2. Security Headers (expanded)")
        try:
            res = self.session.get(self.url, timeout=8)
            headers = res.headers
            self.report['debug']['response_headers'] = dict(headers)

            # list of headers we want to check
            wanted = {
                'Strict-Transport-Security': 'hsts_header',
                'Content-Security-Policy': 'csp_header',
                'X-Content-Type-Options': 'x_content_type_header',
                'X-Frame-Options': 'x_frame_options',
                'Referrer-Policy': 'referrer_policy',
                'Permissions-Policy': 'permissions_policy',
                'X-XSS-Protection': 'x_xss_protection',
                'Cross-Origin-Resource-Policy': 'corp',
                'Cross-Origin-Embedder-Policy': 'coep',
                'Cross-Origin-Opener-Policy': 'coop'
            }

            for hdr, key in wanted.items():
                present = hdr in headers
                # special content checks
                if hdr == 'X-Content-Type-Options':
                    ok = headers.get('X-Content-Type-Options', '').lower() == 'nosniff'
                    self.report['checks'][key] = ok
                    if ok:
                        print_status("X-Content-Type-Options header is 'nosniff'", "PASS")
                    else:
                        print_status("X-Content-Type-Options header is not 'nosniff'", "WARN")
                        self.report['recommendations'].append("Set the X-Content-Type-Options header to 'nosniff'.")
                    continue

                if hdr == 'Content-Security-Policy':
                    if present and headers.get('Content-Security-Policy').strip():
                        self.report['checks'][key] = True
                        print_status("Content-Security-Policy (CSP) header", "PASS")
                    else:
                        self.report['checks'][key] = False
                        print_status("Content-Security-Policy (CSP) header", "WARN")
                        self.report['recommendations'].append("Implement a CSP to mitigate XSS and other injection attacks.")
                    continue

                # default presence check
                if present:
                    self.report['checks'][key] = True
                    print_status(f"{hdr} header", "PASS")
                else:
                    self.report['checks'][key] = False
                    print_status(f"{hdr} header", "WARN")
                    self.report['recommendations'].append(f"Add the {hdr} header.")

            # extra debug: print a short snapshot of headers
            print_status("Response headers snapshot (debug)", "DEBUG")
            # print a few key headers for debug readability
            for k in ['Server', 'Date', 'Content-Type', 'Content-Encoding']:
                if k in headers:
                    print(f"    {k}: {headers[k]}")

        except requests.exceptions.RequestException as e:
            self.report['debug']['headers_exception'] = str(e)
            print_status("Could not fetch headers from the domain", "FAIL")
            self.report["recommendations"].append("Ensure the site is reachable and does not block automated requests.")

    # ---------------- Robots/Sitemaps/AI Declarations ----------------
    def check_robots_and_ai_declarations(self):
        print_subheader("3. Robots, Sitemaps & AI Declarations")
        # robots.txt
        robots_url = f"https://{self.domain}/robots.txt"
        try:
            r = self.session.get(robots_url, timeout=6)
            if r.status_code == 200 and r.text.strip():
                self.report['checks']['robots_txt'] = True
                print_status("robots.txt present", "PASS")
                # debug: show first few lines
                sample = '\n'.join(r.text.strip().splitlines()[:10])
                print_status("robots.txt sample (first lines)", "DEBUG")
                for line in sample.splitlines():
                    print('   ' + line)
            else:
                self.report['checks']['robots_txt'] = False
                print_status("robots.txt not found or empty", "WARN")
                self.report['recommendations'].append("Add a robots.txt to express crawling rules for agents.")
        except requests.exceptions.RequestException as e:
            self.report['checks']['robots_txt'] = False
            self.report['debug']['robots_exception'] = str(e)
            print_status("robots.txt fetch failed", "WARN")

        # ai-robots.txt (AI-specific)
        ai_robots_url = f"https://{self.domain}/ai-robots.txt"
        try:
            r = self.session.get(ai_robots_url, timeout=6)
            if r.status_code == 200 and r.text.strip():
                self.report['checks']['ai_robots_txt'] = True
                print_status("ai-robots.txt present (AI-specialized robots)", "PASS")
            else:
                self.report['checks']['ai_robots_txt'] = False
                print_status("ai-robots.txt not found", "WARN")
        except requests.exceptions.RequestException:
            self.report['checks']['ai_robots_txt'] = False
            print_status("ai-robots.txt fetch failed", "WARN")

        # sitemap at /sitemap.xml
        sitemap_url = f"https://{self.domain}/sitemap.xml"
        try:
            r = self.session.get(sitemap_url, timeout=6)
            if r.status_code == 200 and r.text.strip():
                self.report['checks']['sitemap_root'] = True
                print_status("sitemap.xml found at root", "PASS")
            else:
                self.report['checks']['sitemap_root'] = False
                print_status("sitemap.xml not found at root", "WARN")
        except requests.exceptions.RequestException:
            self.report['checks']['sitemap_root'] = False
            print_status("sitemap.xml fetch failed", "WARN")

        # machine-readable AI declarations: ai.txt, llm.txt, .well-known/ai.json etc.
        declarations = ["/.well-known/ai.json", "/.well-known/llm-policy.json", "/ai.txt", "/llm.txt"]
        for path in declarations:
            url = f"https://{self.domain}{path}"
            try:
                r = self.session.get(url, timeout=6)
                key = path.strip('/').replace('/', '_')
                if r.status_code == 200 and r.text.strip():
                    self.report['checks'][f"decl_{key}"] = True
                    print_status(f"Found machine-readable declaration: {url}", "PASS")
                else:
                    self.report['checks'][f"decl_{key}"] = False
                
            except requests.exceptions.RequestException:
                self.report['checks'][f"decl_{key}"] = False

    # ---------------- DNS & Email Auth ----------------
    def check_domain_identity_records(self):
        print_subheader("4. Domain Identity & Trust (DNS Records & Email Auth)")
        base_domain = self.domain.replace('www.', '')

        if not DNSPYTHON_AVAILABLE:
            print_status("dnspython not installed — skipping CAA/DMARC/SPF/DKIM checks", "WARN")
            self.report['recommendations'].append("Install dnspython: pip install dnspython to enable DNS checks.")
            self.report['debug']['dnspython'] = False
            return

        # CAA
        try:
            answers = dns.resolver.resolve(base_domain, 'CAA')
            self.report["checks"]["caa_record"] = True
            print_status("Certification Authority Authorization (CAA) record", "PASS")
            self.report['debug']['caa'] = [r.to_text() for r in answers]
        except (dns.resolver.NoAnswer, dns.resolver.NXDOMAIN):
            self.report["checks"]["caa_record"] = False
            print_status("Certification Authority Authorization (CAA) record", "WARN")
            self.report["recommendations"].append("Add a CAA DNS record to specify which CAs can issue certificates.")
        except Exception as e:
            self.report["checks"]["caa_record"] = False
            self.report['debug']['caa_exception'] = str(e)
            print_status("CAA check failed", "WARN")

        # DMARC (TXT at _dmarc)
        try:
            answers = dns.resolver.resolve(f'_dmarc.{base_domain}', 'TXT')
            self.report["checks"]["dmarc_record"] = True
            print_status("DMARC email authentication record", "PASS")
            self.report['debug']['dmarc'] = [r.to_text() for r in answers]
        except (dns.resolver.NoAnswer, dns.resolver.NXDOMAIN):
            self.report["checks"]["dmarc_record"] = False
            print_status("DMARC email authentication record", "WARN")
            self.report["recommendations"].append("Add a DMARC DNS record to prevent email spoofing and phishing.")
        except Exception as e:
            self.report["checks"]["dmarc_record"] = False
            self.report['debug']['dmarc_exception'] = str(e)
            print_status("DMARC check failed", "WARN")

        # SPF
        try:
            answers = dns.resolver.resolve(base_domain, 'TXT')
            spf_found = False
            for r in answers:
                txt = r.to_text().strip('"')
                if txt.startswith('v=spf1'):
                    spf_found = True
                    self.report['debug']['spf_txt'] = txt
            self.report['checks']['spf_record'] = spf_found
            if spf_found:
                print_status("SPF record present", "PASS")
            else:
                print_status("SPF record not found", "WARN")
                self.report['recommendations'].append("Add an SPF TXT record to authorize sending mail servers.")
        except Exception as e:
            self.report['checks']['spf_record'] = None
            self.report['debug']['spf_exception'] = str(e)
            print_status("SPF check failed or not available", "WARN")

        # DKIM — requires checking selectors on MX hosts; lightweight approach: attempt to find common selector _domainkey
        try:
            # not doing full selector discovery; just attempt a TXT lookup for default selector 'default._domainkey'
            selector = 'default'
            try:
                answers = dns.resolver.resolve(f'{selector}._domainkey.{base_domain}', 'TXT')
                self.report['checks']['dkim'] = True
                print_status("DKIM selector 'default' found (basic check)", "PASS")
            except Exception:
                self.report['checks']['dkim'] = False
                print_status("DKIM 'default' selector not found (full DKIM discovery not performed)", "WARN")
                self.report['recommendations'].append("Ensure DKIM selectors are published for email signing.")
        except Exception as e:
            self.report['debug']['dkim_exception'] = str(e)

    # ---------------- DNSSEC, WHOIS & Infra ----------------
    def check_dnssec_and_infra(self):
        print_subheader("5. DNSSEC, WHOIS hints & Infrastructure Signals")
        base_domain = self.domain.replace('www.', '')
        if not DNSPYTHON_AVAILABLE:
            print_status("dnspython not installed — skipping DNSSEC checks", "WARN")
            self.report['debug']['dnssec'] = False
        else:
            try:
                # Check for DNSKEY/RRSIG via dns.resolver (best-effort)
                try:
                    dns.resolver.resolve(base_domain, 'DNSKEY')
                    self.report['checks']['dnssec_keys'] = True
                    print_status("DNSSEC DNSKEY found", "PASS")
                except Exception:
                    self.report['checks']['dnssec_keys'] = False
                    print_status("DNSSEC DNSKEY not found", "WARN")

                try:
                    dns.resolver.resolve(base_domain, 'RRSIG')
                    self.report['checks']['dnssec_rrsig'] = True
                    print_status("DNSSEC RRSIG records present", "PASS")
                except Exception:
                    self.report['checks']['dnssec_rrsig'] = False
                    print_status("DNSSEC RRSIG not found", "WARN")
            except Exception as e:
                self.report['debug']['dnssec_exception'] = str(e)

        # Basic resolution and reverse DNS
        try:
            ip = socket.gethostbyname(self.domain)
            self.report['debug']['resolved_ip'] = ip
            print_status(f"Domain resolves to IP: {ip}", "INFO")
            try:
                rev = socket.gethostbyaddr(ip)[0]
                self.report['debug']['reverse_dns'] = rev
                print_status(f"Reverse DNS: {rev}", "DEBUG")
            except Exception as e:
                self.report['debug']['reverse_dns_error'] = str(e)
        except Exception as e:
            self.report['debug']['resolution_error'] = str(e)
            print_status("Domain name resolution failed", "WARN")

        # WHOIS hint: we avoid adding a dependency but offer debug instruction
        print_status("WHOIS/domain age check", "DEBUG")
        self.report['debug']['whois_hint'] = "Run 'whois <domain>' or use python-whois for automated checks."

    # ---------------- Availability & Crawl-Friendliness ----------------
    def check_availability_and_crawl(self):
        print_subheader("6. Availability, Rate Limits & Crawl-Friendliness")
        # HEAD request
        try:
            r = self.session.head(self.url, timeout=6)
            self.report['debug']['head_status'] = r.status_code
            if r.status_code < 400:
                print_status("HEAD request succeeded", "PASS")
            else:
                print_status(f"HEAD returned status {r.status_code}", "WARN")
            # compression
            ce = r.headers.get('Content-Encoding') or r.headers.get('content-encoding')
            if ce:
                print_status(f"Content-Encoding: {ce}", "INFO")
        except Exception as e:
            self.report['debug']['head_exception'] = str(e)
            print_status("HEAD request failed", "WARN")

    # ---------------- Report generation ----------------
    def _generate_final_report(self):
        score = 0
        if self.report["checks"].get("http_redirect"): score += 15
        if self.report["checks"].get("ssl_valid"): score += 15
        if self.report["checks"].get("hsts_header"): score += 10
        if self.report["checks"].get("csp_header"): score += 10
        if self.report["checks"].get("x_content_type_header"): score += 5
        if self.report["checks"].get("caa_record"): score += 10
        if self.report["checks"].get("dmarc_record"): score += 5
        if self.report["checks"].get("robots_txt"): score += 5
        if self.report["checks"].get("ai_robots_txt"): score += 5
        if self.report["checks"].get("decl_.well-known_ai.json"):
            score += 5
        # cap and status
        self.report["score"] = min(100, score)
        if self.report["score"] >= 90: self.report["status"] = "Excellent"
        elif self.report["score"] >= 70: self.report["status"] = "Good"
        elif self.report["score"] >= 40: self.report["status"] = "Needs Improvement"
        else: self.report["status"] = "Poor"

    def _print_final_report(self):
        print_header("Final Assessment Report")
        print_status("Overall Status", self.report['status'])
        print_status("ARI Score (out of 100)", self.report['score'])

        if self.report["recommendations"]:
            print_subheader("Recommendations")
            for rec in self.report["recommendations"]:
                print_recommendation(rec)

        # Key debug summary
        print_subheader("Key Check Summary (debug)")
        keys = ['http_redirect', 'ssl_valid', 'hsts_header', 'csp_header', 'caa_record', 'dmarc_record', 'dnssec_keys', 'robots_txt', 'ai_robots_txt']
        for k in keys:
            print(f"  - {k:22}: {self.report['checks'].get(k)}")

    # ---------------- Run ----------------
    def run_analysis(self):
        print_header("ARI Sub-Pillar 1.6: Domain Security & Identity Trust (Patched A1)")
        self.check_https_enforcement()
        self.check_security_headers()
        self.check_robots_and_ai_declarations()
        self.check_domain_identity_records()
        self.check_dnssec_and_infra()
        self.check_availability_and_crawl()
        self._generate_final_report()
        self._print_final_report()







In [18]:
target = input("Enter URL :::: ").strip()
a = DomainTrustAnalyzer(target)
a.run_analysis()
print("\n" + "="*60)
print("RAW REPORT OBJECT")
print("="*60)
print(a.report)

Enter URL ::::  https://buildbridges.co/



 ARI Sub-Pillar 1.6: Domain Security & Identity Trust (Patched A1)

----------------------------------------------------------------------
 1. HTTPS Enforcement & Certificate
----------------------------------------------------------------------
HTTP requests redirect to HTTPS                         [PASS]
SSL Certificate retrieved and parsed                    [PASS]
Certificate is valid for 79 more days                   [PASS]
Certificate SANs: ['buildbridges.co', 'www.buildbridges.co'] [DEBUG]

----------------------------------------------------------------------
 2. Security Headers (expanded)
----------------------------------------------------------------------
Strict-Transport-Security header                        [WARN]
Content-Security-Policy (CSP) header                    [WARN]
X-Content-Type-Options header is not 'nosniff'          [WARN]
X-Frame-Options header                                  [WARN]
Referrer-Policy header                                  [WARN]
Permi